<a href="https://colab.research.google.com/github/AyushiM1102/Electra_classification_fake_vs_real_news/blob/main/electra_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Reference tutorial:
# https://velog.io/@na2na8/ELECTRA%EB%A1%9C-Binary-Classification#electra-with-pytorch-lightning

In [2]:
!pip install transformers --quiet

     |████████████████████████████████| 4.2 MB 9.2 MB/s 
     |████████████████████████████████| 596 kB 46.4 MB/s 
     |████████████████████████████████| 86 kB 3.1 MB/s 
     |████████████████████████████████| 6.6 MB 46.6 MB/s 


In [3]:
!pip install git+https://github.com/PyTorchLightning/pytorch-lightning --quiet
import pytorch_lightning as pl
print(pl.__version__)

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 418 kB 7.6 MB/s 
     |████████████████████████████████| 140 kB 68.4 MB/s 
     |████████████████████████████████| 1.1 MB 47.9 MB/s 
     |████████████████████████████████| 144 kB 19.3 MB/s 
     |████████████████████████████████| 271 kB 76.7 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
1.7.0dev


In [4]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 9.1 MB/s 
     |████████████████████████████████| 181 kB 67.0 MB/s 
     |████████████████████████████████| 145 kB 51.5 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=1d66d329ad12a5b3033839ab12c4ff1b698c7852c89412e5189daf393fd7b7c3
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [5]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
import os
import re

import numpy as np
import pandas as pd

import torch
import torchmetrics
import torch.nn as nn
import wandb
from torch.utils.data import DataLoader, Dataset

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import loggers as pl_loggers
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import transformers
from transformers import ElectraForSequenceClassification, ElectraTokenizer, AdamW

device = torch.device("cuda")


In [7]:
!git clone https://github.com/AyushiM1102/Electra_classification_fake_vs_real_news.git

Cloning into 'Electra_classification_fake_vs_real_news'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 81 (delta 2), reused 0 (delta 0), pack-reused 75
Unpacking objects: 100% (81/81), done.


In [8]:
!unzip /content/Electra_classification_fake_vs_real_news/data/WELFake_Dataset.csv.zip -d /content/Electra_classification_fake_vs_real_news/dataset

Archive:  /content/Electra_classification_fake_vs_real_news/data/WELFake_Dataset.csv.zip
  inflating: /content/Electra_classification_fake_vs_real_news/dataset/WELFake_Dataset.csv  
  inflating: /content/Electra_classification_fake_vs_real_news/dataset/__MACOSX/._WELFake_Dataset.csv  


In [9]:
# Overview of database
# df = pd.read_csv('/content/Electra_classification_fake_vs_real_news/sample_dataset/train.csv', sep=',')
datapath = f'/content/Electra_classification_fake_vs_real_news/dataset/WELFake_Dataset.csv'
df = pd.read_csv(datapath, sep=',')
df = df.dropna(axis=0)
df.drop_duplicates(inplace=True)

In [10]:
df['label'].value_counts()

1    36509
0    35028
Name: label, dtype: int64

In [11]:
train_size = int(0.7 * len(df))
val_size = int((2/3)*(len(df) - train_size))
test_size = int((1/3)*(len(df) - train_size))
train_size, val_size, test_size

(50075, 14308, 7154)

In [12]:
train_size + val_size + test_size

71537

In [13]:
dataset = shuffle(df)

In [18]:
train_dataset, general_test_dataset  = train_test_split(dataset, train_size=int(0.7 * len(dataset)), test_size=int(0.3 * len(dataset)))

In [19]:
val_dataset, test_dataset = train_test_split(general_test_dataset, train_size=int((2/3) * len(general_test_dataset)), test_size=int((1/3) * len(general_test_dataset)))

In [20]:
len(train_dataset),len(val_dataset),len(test_dataset)

(50075, 14307, 7153)

In [ ]:
train_dataset.to_csv('/content/Electra_classification_fake_vs_real_news/dataset/train.csv', index = False)
val_dataset.to_csv('/content/Electra_classification_fake_vs_real_news/dataset/val.csv', index = False)
test_dataset.to_csv('/content/Electra_classification_fake_vs_real_news/dataset/test.csv', index = False)

In [ ]:
class ElectraClassificationDataset(Dataset) :
    def __init__(self, path, sep, doc_col, label_col, max_length, num_workers=1, labels_dict=None) :

        self.tokenizer = ElectraTokenizer.from_pretrained("google/electra-small-discriminator")
        self.max_length = max_length
        self.doc_col = doc_col
        self.label_col = label_col

        # labels, ex : {True : 1, False : 0}
        self.labels_dict = labels_dict

        # dataset
        df = pd.read_csv(path, sep=sep)
        df = df.dropna(axis=0)
        df.drop_duplicates(subset=[self.doc_col], inplace=True)
        self.dataset = df

    def __len__(self) :
        return len(self.dataset)
    
    # Clean text
    def cleanse(self, text) :
        url_pattern = re.compile(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')
        processed = url_pattern.sub(' ', text)
        processed = processed.replace('#', '')
        processed = processed.replace('@', '')
        processed = processed.strip()
        return processed

    def __getitem__(self, idx) :
        document = self.cleanse(self.dataset[self.doc_col].iloc[idx])
        #print(document)
        inputs = self.tokenizer(
            document,
            return_tensors='pt',
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            add_special_tokens=True
        )

        if self.labels_dict :
            label = self.labels_dict[self.dataset[self.label_col].iloc[idx]]
        else :
            label = self.dataset[self.label_col].iloc[idx]

        return {
            'input_ids' : inputs['input_ids'][0],
            'attention_mask' : inputs['attention_mask'][0],
            'label' : int(label)
        }

In [ ]:
class ElectraClassificationDataModule(pl.LightningDataModule) :
    def __init__(self, train_path, valid_path, test_path, max_length, batch_size, sep,
                doc_col, label_col, num_workers=1, labels_dict=None) :
        super().__init__()
        self.batch_size = batch_size
        self.train_path = train_path
        self.valid_path = valid_path
        self.test_path = test_path
        self.max_length = max_length
        self.doc_col = doc_col
        self.label_col = label_col
        self.sep = sep
        self.num_workers = num_workers
        self.labels_dict = labels_dict

    def setup(self, stage=None) :
        self.set_train = ElectraClassificationDataset(self.train_path, sep=self.sep,
                                            doc_col=self.doc_col, label_col=self.label_col,
                                            max_length = self.max_length, labels_dict=self.labels_dict)
        self.set_valid = ElectraClassificationDataset(self.valid_path, sep=self.sep,
                                            doc_col=self.doc_col, label_col=self.label_col,
                                            max_length = self.max_length, labels_dict=self.labels_dict)
        self.set_test = ElectraClassificationDataset(self.test_path, sep=self.sep,
                                            doc_col=self.doc_col, label_col=self.label_col,
                                            max_length = self.max_length, labels_dict=self.labels_dict)
        

    def train_dataloader(self) :
        train = DataLoader(self.set_train, batch_size=self.batch_size, num_workers=self.num_workers, shuffle=True)
        return train
    
    def val_dataloader(self) :
        val = DataLoader(self.set_valid, batch_size=self.batch_size, num_workers=self.num_workers, shuffle=False)
        return val
    
    def test_dataloader(self) :
        test = DataLoader(self.set_test, batch_size=self.batch_size, num_workers=self.num_workers, shuffle=False)
        return test

In [ ]:
# https://medium.com/huggingface/multi-label-text-classification-using-bert-the-mighty-transformer-69714fa3fb3d
# https://huggingface.co/docs/transformers/v4.15.0/en/model_doc/electra#transformers.ElectraForSequenceClassification

In [ ]:
class ElectraClassification(pl.LightningModule) :
    def __init__(self, learning_rate) :
        super().__init__()
        self.learning_rate = learning_rate
        self.save_hyperparameters()
        self.electra = ElectraForSequenceClassification.from_pretrained("google/electra-small-discriminator")

        self.metric_acc = torchmetrics.Accuracy()
        self.metric_f1 = torchmetrics.F1Score(num_classes=2)
        self.metric_rec = torchmetrics.Recall(num_classes=2)
        self.metric_pre = torchmetrics.Precision(num_classes=2)

        self.loss_func = nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, labels=None) :
        output = self.electra(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return output

    def training_step(self, batch, batch_idx) :
        '''
        ##########################################################
        electra forward input shape information
        * input_ids.shape (batch_size, max_length)
        * attention_mask.shape (batch_size, max_length)
        * label.shape (batch_size,)
        ##########################################################
        '''

        # change label shape (list -> torch.Tensor((batch_size, 1)))
        label = batch['label'].view([-1,1])

        output = self(input_ids=batch['input_ids'].to(device),
                        attention_mask=batch['attention_mask'].to(device),
                        labels=label.to(device))
        '''
        ##########################################################
        electra forward output shape information
        * loss.shape (1,)
        * logits.shape (batch_size, config.num_labels=2)
        '''
        logits = output.logits

        loss = output.loss
        # loss = self.loss_func(logits.to(device), batch['label'].to(device))

        softmax = nn.functional.softmax(logits, dim=1)
        preds = softmax.argmax(dim=1)

        self.log("train_loss", loss, prog_bar=True)
        
        return {
            'loss' : loss,
            'pred' : preds,
            'label' : batch['label']
        }

    def training_epoch_end(self, outputs, state='train') :
        y_true = []
        y_pred = []
        for i in outputs :
            y_true += i['label'].tolist()
            y_pred += i['pred'].tolist()

        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred)
        rec = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        # self.log(state+'_acc', acc, on_epoch=True, prog_bar=True)
        # self.log(state+'_precision', prec, on_epoch=True, prog_bar=True)
        # self.log(state+'_recall', rec, on_epoch=True, prog_bar=True)
        # self.log(state+'_f1', f1, on_epoch=True, prog_bar=True)
        print(f'[Epoch {self.trainer.current_epoch} {state.upper()}] Acc: {acc}, Prec: {prec}, Rec: {rec}, F1: {f1}')

    def validation_step(self, batch, batch_idx) :
        '''
        ##########################################################
        electra forward input shape information
        * input_ids.shape (batch_size, max_length)
        * attention_mask.shape (batch_size, max_length)
        ##########################################################
        '''
        output = self(input_ids=batch['input_ids'].to(device),
                        attention_mask=batch['attention_mask'].to(device))
        logits = output.logits
        preds = nn.functional.softmax(logits, dim=1).argmax(dim=1)
        labels = batch['label']
        accuracy = self.metric_acc(preds, labels)
        f1 = self.metric_f1(preds, labels)
        recall = self.metric_rec(preds, labels)
        precision = self.metric_pre(preds, labels)
        self.log('val_accuracy', accuracy, on_epoch=True, prog_bar=True)
        self.log('val_f1', f1, on_epoch=True, prog_bar=True)
        self.log('val_recall', recall, on_epoch=True, prog_bar=True)
        self.log('val_precision', precision, on_epoch=True, prog_bar=True)
        return {
            'accuracy' : accuracy,
            'f1' : f1,
            'recall' : recall,
            'precision' : precision
        }

    def validation_epoch_end(self, outputs) :
        val_acc = torch.stack([i['accuracy'] for i in outputs]).mean()
        val_f1 = torch.stack([i['f1'] for i in outputs]).mean()
        val_rec = torch.stack([i['recall'] for i in outputs]).mean()
        val_pre = torch.stack([i['precision'] for i in outputs]).mean()
        # self.log('val_f1', val_f1, on_epoch=True, prog_bar=True)
        # self.log('val_acc', val_acc, on_epoch=True, prog_bar=True)
        print(f'val_accuracy : {val_acc}, val_f1 : {val_f1}, val_recall : {val_rec}, val_precision : {val_pre}')
    
    def test_step(self, batch, batch_idx):
        output = self(input_ids=batch['input_ids'].to(device),
                        attention_mask=batch['attention_mask'].to(device))
        logits = output.logits
        preds = nn.functional.softmax(logits, dim=1).argmax(dim=1)
        labels = batch['label']
        accuracy = self.metric_acc(preds, labels)
        f1 = self.metric_f1(preds, labels)
        recall = self.metric_rec(preds, labels)
        precision = self.metric_pre(preds, labels)
        self.log('test_accuracy', accuracy, on_epoch=True, prog_bar=True)
        self.log('test_f1', f1, on_epoch=True, prog_bar=True)
        self.log('test_recall', recall, on_epoch=True, prog_bar=True)
        self.log('test_precision', precision, on_epoch=True, prog_bar=True)

        return {
            'accuracy' : accuracy,
            'f1' : f1,
            'recall' : recall,
            'precision' : precision
        }


    def test_end(self, outputs):
        test_acc = torch.stack([i['accuracy'] for i in outputs]).mean()
        test_f1 = torch.stack([i['f1'] for i in outputs]).mean()
        test_rec = torch.stack([i['recall'] for i in outputs]).mean()
        test_pre = torch.stack([i['precision'] for i in outputs]).mean()
        # self.log('val_f1', val_f1, on_epoch=True, prog_bar=True)
        # self.log('val_acc', val_acc, on_epoch=True, prog_bar=True)
        print(f'test_accuracy : {test_acc}, test_f1 : {test_f1}, test_recall : {test_rec}, test_precision : {test_pre}')
        

    # def test_epoch_end(self, outputs):
    #     all_preds, all_labels = [], []
    #     for output in outputs:
    #         probs = list(output['logits'].cpu().detach().numpy()) # predicted values
    #         labels = list(output['labels'].flatten().cpu().detach().numpy())
    #         all_preds.extend(probs)
    #         all_labels.extend(labels)

    #     # you can calculate R2 here or save results as file
    #     r2 = ...
    
    # def predict_step(self, test_batch):
    #   x, y = test_batch
    #   logits = self.forward(x)
    #   return {'logits': logits, 'labels':y}

    def configure_optimizers(self) :
        optimizer = torch.optim.AdamW(self.electra.parameters(), lr=self.learning_rate)
        lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
        
        return {
            'optimizer' : optimizer,
            'lr_scheduler' : lr_scheduler
        }

In [ ]:
df

In [ ]:
# Main to train the model

# Initialize WandB 
wandb_logger = WandbLogger(project='Electra Classification', 
                           log_model='all')
model = ElectraClassification(learning_rate=0.0001)

wandb.watch(model)

dm = ElectraClassificationDataModule(batch_size=8, train_path='/content/Electra_classification_fake_vs_real_news/dataset/train.csv', valid_path='/content/Electra_classification_fake_vs_real_news/dataset/val.csv',
                                     test_path='/content/Electra_classification_fake_vs_real_news/dataset/test.csv',
                                max_length=512, sep=',', doc_col='text', label_col='label', num_workers=1)
dm.setup()
train_dataset = dm.train_dataloader()
valid_dataset = dm.val_dataloader()

checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor='val_accuracy',
                                                dirpath='./sample_electra_binary_nsmc_chpt',
                                                filename='ELECTRA/{epoch:02d}-{val_accuracy:.3f}',
                                                verbose=True,
                                                save_last=True,
                                                mode='max',
                                                save_top_k=-1,
                                                )

tb_logger = pl_loggers.TensorBoardLogger(os.path.join('./sample_electra_binary_nsmc_chpt', 'tb_logs'))

lr_logger = pl.callbacks.LearningRateMonitor()

trainer = pl.Trainer(
    default_root_dir='./sample_electra_binary_nsmc_chpt/checkpoints',
    logger = wandb_logger,
    callbacks = [checkpoint_callback, lr_logger],
    max_epochs=5,
    gpus=1)

trainer.fit(model, train_dataset, valid_dataset)

In [ ]:
test_dataset = dm.test_dataloader()
# trainer.test(dataloaders=test_dataset)
trainer.test(ckpt_path=None)

In [ ]:
wandb.finish()

### Code below this line is extraneous. 

In [ ]:
electra = ElectraForSequenceClassification.from_pretrained("google/electra-small-discriminator")

# Check parameters
dm = ElectraClassificationDataModule(batch_size=8, train_path='/content/Electra_classification_fake_vs_real_news/sample_dataset/train.csv', valid_path='/content/Electra_classification_fake_vs_real_news/sample_dataset/val.csv',
                                    max_length=256, sep=',', doc_col='Tweet', label_col='is_retweet', num_workers=1)

dm.setup()

t = dm.train_dataloader()

print(t)
for idx, data in enumerate(t):
    print(idx, data['input_ids'].shape, data['attention_mask'].shape, data['label'].shape)

# Concatenate the batches ?? ********* PENDING *********** HOW TO DO THIS ?? 
#idx, data = enumerate(t)

v = dm.val_dataloader()

for idx, data in enumerate(v) :
  print(idx, data['input_ids'].shape, data['attention_mask'].shape, data['label'].shape)
  # print(idx, data['input_ids'], data['attention_mask'], data['label'])

  output = electra.forward(data['input_ids'], attention_mask=data['attention_mask'], labels=data['label'].view([-1,1]))

  print("This is the loss")
  print(output.loss)
  # print(output.loss.shape)
  # print(output.logits)
  print(output.logits.shape)

  softmax = nn.functional.softmax(output.logits, dim=1)
  print('softmax', softmax)
  pred = softmax.argmax(dim=1)
  print('pred', pred)

  y_true = data['label'].tolist()
  y_pred = pred.tolist()

acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred)
rec = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f'acc : {acc}, prec : {prec}, rec : {rec}, f1 : {f1}')
